In [ ]:
# this file is try to cross validate between data sources

In [1]:
import os, ast, json, uuid, re
from itertools import combinations
from scipy import stats
import numpy as np
import pandas as pd
import geopandas

import lxml
from lxml import etree
from lxml.etree import QName
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.colors import n_colors
import plotly.io as pio
pio.renderers.default = "browser"
print(pio.renderers)

Renderers configuration
-----------------------
    Default renderer: 'browser'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [2]:
pd.set_option('display.max_columns', None)

In [3]:
dist_csv_path = 'C:/Users/user/Downloads/dist-rep-wran-ampcode-v4 (1).csv'
ws_csv_path = 'C:/Users/user/Downloads/ws_pre_license.csv'

In [7]:
xml_path = 'D:/Onedrive/SynMAR2020/แบ่งเขต.xml'
dist_csv_path = "C:/Users/ASUS/Downloads/dist-rep-wran-ampcode-v4.csv"

In [18]:
namespaces = {
    'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
    'x': 'urn:schemas-microsoft-com:office:excel',
    'o': 'urn:schemas-microsoft-com:office:office',
    'html': 'http://www.w3.org/TR/REC-html40',
    '': 'urn:schemas-microsoft-com:office:spreadsheet' # Default namespace
}

In [4]:
dist_df = pd.read_csv(dist_csv_path)

In [61]:
dist_df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)

In [6]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
0,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205,2024-01-31,ศรีประจันต์,7205,0.25,125.00
1,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302,2024-01-31,กะทู้,8302,0.50,0.00
2,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005,2024-01-31,บ้านโป่ง,7005,0.50,306.00
3,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.25,116.25
4,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.50,478.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601,2024-12-31,เมืองนราธิวาส,9601,1.00,342.00
28175,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0

In [5]:
ws_df = pd.read_csv(ws_csv_path)

In [ ]:
# before cleanse

In [10]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3885 non-null   object 
 1   ชื่อสถานที่          3885 non-null   object 
 2   ที่อยู่สถานที่       3885 non-null   object 
 3   บ้านเลขที่           3872 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 957 non-null    object 
 6   อาคาร                531 non-null    object 
 7   ซอย                  1314 non-null   object 
 8   ถนน                  3188 non-null   object 
 9   หมู่                 529 non-null    object 
 10  ตำบล                 3881 non-null   object 
 11  อำเภอ                3881 non-null   object 
 12  จังหวัด              3881 non-null   object 
 13  รหสไปรษณีย์          3881 non-null   float64
 14  เบอร์โทรศัพท์        3807 non-null   object 
 15  เบอร์โทรสาร          642 non-null    o

<h1>Parsing XML in case parsed csv is being used skip this part</h1>

In [62]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
0,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
1,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
2,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
3,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
4,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
28175,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxsz

In [7]:
tree = etree.parse(xml_path)

In [8]:
root = tree.getroot()

In [12]:
root.findall('.//ss:Worksheet', namespaces)

[<Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a2d6940>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cb80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cdc0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ce40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cd80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ca80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16da40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16d240>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cec0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cf80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16c800>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet

In [19]:

ss_name_qn = QName(namespaces['ss'], 'Name')
ss_row_qn = QName(namespaces['ss'], 'Row')
ss_cell_qn = QName(namespaces['ss'], 'Cell')
ss_data_qn = QName(namespaces['ss'], 'Data')
ss_type_qn = QName(namespaces['ss'], 'Type')

In [16]:
worksheet_names = []
for worksheet in root.findall('.//ss:Worksheet', namespaces):
    name = worksheet.get(ss_name)
    if name:
        worksheet_names.append(name)

In [21]:
namespaces

{'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
 'x': 'urn:schemas-microsoft-com:office:excel',
 'o': 'urn:schemas-microsoft-com:office:office',
 'html': 'http://www.w3.org/TR/REC-html40',
 '': 'urn:schemas-microsoft-com:office:spreadsheet'}

In [25]:
len(root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn))

3886

In [26]:
first_ws_allrows = root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn)

In [27]:
table_data = []
for row in first_ws_allrows:
    row_data = []
    # Iterate through all Cell elements in the Row
    for cell in row.findall(ss_cell_qn):
        # Find the Data element within the Cell
        data_element = cell.find(ss_data_qn)
        if data_element is not None:
            # Get the data type and text content
            # The get() method returns a Unicode string, which is what we want
            data_value = data_element.text
            row_data.append(data_value)
        else:
            # Append an empty string or None for empty cells
            row_data.append(None) 
    table_data.append(row_data)

In [29]:
# Create the pandas DataFrame from the extracted data
df = pd.DataFrame(table_data)

# The first row often contains headers. You can set them as column names.
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ชื่อผู้รับอนุญาต     3885 non-null   object
 1   ชื่อสถานที่          3885 non-null   object
 2   ที่อยู่สถานที่       3885 non-null   object
 3   บ้านเลขที่           3872 non-null   object
 4   ห้อง                 281 non-null    object
 5   ชั้น                 957 non-null    object
 6   อาคาร                531 non-null    object
 7   ซอย                  1314 non-null   object
 8   ถนน                  3188 non-null   object
 9   หมู่                 529 non-null    object
 10  ตำบล                 3881 non-null   object
 11  อำเภอ                3881 non-null   object
 12  จังหวัด              3881 non-null   object
 13  รหสไปรษณีย์          3881 non-null   object
 14  เบอร์โทรศัพท์        3807 non-null   object
 15  เบอร์โทรสาร          642 non-null    object
 16  ชื่อผู

In [42]:
df.loc[df['ประเภทคำขอ'].str.contains('ขาย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)                542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)              359
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                             62
ขาย วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์      1
Name: count, dtype: int64

In [43]:
df.loc[df['ประเภทคำขอ'].str.contains('จำหน่าย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ค.)        759
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ก.)         60
จำหน่าย  ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.)                                  15
จำหน่าย ยส 2 เพื่อการศึกษาวิจัยทางด้านการแพทย์หรือทางด้านวิทยาศาสตร์ (หมวด ก.)      1
Name: count, dtype: int64

In [17]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วจ2')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ2 สถานพยาบาล เอกชน          147
ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน      4
ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์      1
Name: count, dtype: int64

In [50]:
df.loc[df['ประเภทคำขอ'].str.contains('วัตถุออกฤท')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)      542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)    359
นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์      96
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                   62
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์        18
ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์        2
ส่งออก วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์       1
Name: count, dtype: int64

In [51]:
df.loc[df['ประเภทคำขอ'].str.contains('ผลิต')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
เพื่อผลิต ยส 2                                                                             52
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ค.)                                                      47
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์                  18
ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์    12
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ก.)                                                       2
Name: count, dtype: int64

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ชื่อผู้รับอนุญาต     3885 non-null   object
 1   ชื่อสถานที่          3885 non-null   object
 2   ที่อยู่สถานที่       3885 non-null   object
 3   บ้านเลขที่           3872 non-null   object
 4   ห้อง                 281 non-null    object
 5   ชั้น                 957 non-null    object
 6   อาคาร                531 non-null    object
 7   ซอย                  1314 non-null   object
 8   ถนน                  3188 non-null   object
 9   หมู่                 529 non-null    object
 10  ตำบล                 3881 non-null   object
 11  อำเภอ                3881 non-null   object
 12  จังหวัด              3881 non-null   object
 13  รหสไปรษณีย์          3881 non-null   object
 14  เบอร์โทรศัพท์        3807 non-null   object
 15  เบอร์โทรสาร          642 non-null    object
 16  ชื่อผู

In [52]:
df['จังหวัด'].value_counts().reset_index()

,จังหวัด,count
0,กรุงเทพมหานคร,3477
1,สมุทรปราการ,74
2,ปทุมธานี,63
3,นนทบุรี,60
4,เชียงใหม่,28
5,ชลบุรี,20
6,สมุทรสาคร,14
7,นครปฐม,12
8,สงขลา,11
9,นครราชสีมา,11


In [54]:
df['เลขใบอนุญาต'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขใบอนุญาต,count
0,2/2564,5
1,1/2564,5
2,4/2564,4
3,5/2564,4
4,6/2564,4
...,...,...
3358,ควจ4-ร 19/2568,1
3359,ควจ4-ว 1/2568,1
3360,ควจ4-ร 23/2568,1
3361,ควจ4-ร 21/2568,1


In [57]:
df['เลขรับ'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขรับ,count
0,0,240
7,6800077,1
3608,5900758,1
23,5902038,1
3624,5902037,1
...,...,...
19,6800017,1
20,6800024,1
21,6800025,1
22,6800030,1


In [56]:
df.loc[df['เลขใบอนุญาต'] == '1/2564']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
1817,นางสาว พรพิศ นคเรศไอศูรย์,เมดพลัสคลินิกเวชกรรม,บ้านเลขที่900/31 ห้องเลขที่ี G5 ชั้น G อาคารเอ...,900/31,ห้องเลขที่ี G5,ชั้น G,อาคารเอสวีซิตี้ อาคาร 12,None,พระราม 3,None,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,10120,06 4556 5424,None,นางสาว พรพิศ นคเรศไอศูรย์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6400020,1/2564,อนุมัติ,None
1847,บริษัท โรงพยาบาล ซีจีเอช ลำลูกกา จำกัด,โรงพยาบาลซีจีเอช ลำลูกกา โรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่80/77-80 ถนน หมู่5 ตำบลลำลูกกา อ...,80/77-80,None,None,None,None,None,5,ลำลูกกา,ลำลูกกา,ปทุมธานี,12150,02 088 0888,02 088 0899,พลอากาศโท วันชัย ศิริเสรีวรรณ,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6400392,1/2564,อนุมัติ,None
1979,บริษัท พีลาทัส มารีน จำกัด,บริษัท พีลาทัส มารีน จำกัด,บ้านเลขที่84/1-4 ชั้น 1-4 อาคารยังเพลส แกรนด์...,84/1-4,None,ชั้น 1-4,อาคารยังเพลส แกรนด์ เลอ จาแด๊งน์,พหลโยธิน 37,None,None,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900,0 2930 1801-9,None,นาย วราวิช ฉิมตะวัน,จำหน่าย ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.),6400812,1/2564,อนุมัติ,None
2004,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บ้านเลขที่127 ห้อง 25.02 ชั้น 25 อาคาร เกษรทาว...,127,ห้อง 25.02,ชั้น 25,อาคาร เกษรทาวเวอร์,None,ราชดำริ,None,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,0 2073 0224,None,นางสาว สุทัชชา อัจฉริยวงศ์กุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,6400866,1/2564,อนุมัติ,None
2010,องค์การเภสัชกรรม,องค์การเภสัชกรรม,บ้านเลขที่ 75/1 ซอย ถนนพระราม 6 หมู่ ตำบลทุ...,75/1,None,None,None,None,พระราม 6,None,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400,0 2203 8600-3,None,นาย วิฑูรย์ ด่านวิบูลย์,เพื่อผลิต ยส 2,6400868,1/2564,อนุมัติ,None


In [63]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
0,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
1,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
2,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
3,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
4,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
28175,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxsz

In [77]:
dist_df['CustomerAddressProvinceName'].value_counts()

CustomerAddressProvinceName
กรุงเทพมหานคร    9243
ชลบุรี           1271
นนทบุรี          1208
เชียงใหม่        1062
สมุทรปราการ       874
                 ... 
สุโขทัย            48
ยโสธร              36
มหาสารคาม          35
กาฬสินธุ์          32
สตูล               12
Name: count, Length: 76, dtype: int64

In [79]:
loc_ref_pvhos_bkk

23                                แอ็บส์เมดิคคลินิกเวชกรรม
30       เพื่อนใจคลินิกเฉพาะทางด้านเวชกรรมจิตเวชเด็กและ...
44                               สหแพทย์รัชดาคลินิกเวชกรรม
58                                       มณฑลคลินิกเวชกรรม
65                              คลินิกเวชกรรมแพทย์ลาดพร้าว
                               ...                        
26138                                    ปีติคลินิกเวชกรรม
26262                                    ไวทัลไลฟ์สหคลินิก
26487                          คลินิกเวชกรรมดุสิตาการแพทย์
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
Name: ReferCustomerName, Length: 226, dtype: object

In [69]:
loc_ref_pvhos

1                        คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช
5                           คลินิกเวชกรรมแพทย์หญิงกนกนันท์
6                                    อนุรักษ์คลินิกเวชกรรม
17                                     สีชมพูคลินิกเวชกรรม
21       เอส ไมนด์ คลินิกเฉพาะทางด้านเวชกรรมสาขาจิตเวชศ...
                               ...                        
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
26898           คลินิกเฉพาะทางด้านเวชกรรมจิตเวชแฮปปี้มายด์
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
28028                             คลินิกเวชกรรมดิเรก-พรรณี
28172                                   หมอกิ๊ฟลันสหคลินิก
Name: ReferCustomerName, Length: 611, dtype: object

In [72]:
ref_pvhos_df = pd.DataFrame(loc_ref_pvhos)

In [81]:
ref_pvhos_bkk_df = pd.DataFrame(loc_ref_pvhos_bkk)

In [73]:
ref_pvhos_df['isPresent'] = isPresent

In [82]:
ref_pvhos_bkk_df['isPresent'] = isPresent

In [85]:
ref_pvhos_bkk_df.loc[ref_pvhos_bkk_df['isPresent'] == False].loc[599]

ReferCustomerName    ่คลินิกเวชกรรมแพทย์อิทธิพล
isPresent                                 False
Name: 599, dtype: object

In [86]:
df.loc[df['ชื่อสถานที่'].str.contains('แพทย์อิทธิพล')]

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
914,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6100776,625/2561,อนุมัติ,None
1919,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6400652,ควจ4-ร 14/2564,อนุมัติ,None


In [87]:
df.to_csv("C:/Users/ASUS/Downloads/ws_pre_license.csv", index=False)

<h1>Cleansing steps</h1>

<h2>drop duplicates records</h2>

In [ ]:
# before

In [6]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3810 non-null   object 
 1   ชื่อสถานที่          3810 non-null   object 
 2   ที่อยู่สถานที่       3810 non-null   object 
 3   บ้านเลขที่           3797 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 942 non-null    object 
 6   อาคาร                522 non-null    object 
 7   ซอย                  1293 non-null   object 
 8   ถนน                  3162 non-null   object 
 9   หมู่                 489 non-null    object 
 10  ตำบล                 3806 non-null   object 
 11  อำเภอ                3806 non-null   object 
 12  จังหวัด              3806 non-null   object 
 13  รหสไปรษณีย์          3806 non-null   float64
 14  เบอร์โทรศัพท์        3732 non-null   object 
 15  เบอร์โทรสาร          630 non-null    o

In [7]:
ws_df = ws_df.drop_duplicates()

In [8]:
# after
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3810 non-null   object 
 1   ชื่อสถานที่          3810 non-null   object 
 2   ที่อยู่สถานที่       3810 non-null   object 
 3   บ้านเลขที่           3797 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 942 non-null    object 
 6   อาคาร                522 non-null    object 
 7   ซอย                  1293 non-null   object 
 8   ถนน                  3162 non-null   object 
 9   หมู่                 489 non-null    object 
 10  ตำบล                 3806 non-null   object 
 11  อำเภอ                3806 non-null   object 
 12  จังหวัด              3806 non-null   object 
 13  รหสไปรษณีย์          3806 non-null   float64
 14  เบอร์โทรศัพท์        3732 non-null   object 
 15  เบอร์โทรสาร          630 non-null    o

In [ ]:
# replaced file

In [16]:
ws_df.to_csv(ws_csv_path,index=False)

<h2>filter relevant data</h2>

In [11]:
type_keywords = ['วัตถุออกฤทธิ์ในประเภท 2', 'วจ 2', 'วจ2','ครอบครอง วจ ไม่ใช่สถานพยาบาล เอกชน','ครอบครอง วจ สถานพยาบาล เอกชน']

In [12]:
wsf_df = pd.DataFrame()
for t in type_keywords:
    loc_df = ws_df.loc[ws_df['ประเภทคำขอ'].str.contains(t)]
    wsf_df = pd.concat([wsf_df,loc_df], axis=0).drop_duplicates()

In [41]:
wsf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1344 entries, 85 to 76
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     1344 non-null   object 
 1   ชื่อสถานที่          1344 non-null   object 
 2   ที่อยู่สถานที่       1344 non-null   object 
 3   บ้านเลขที่           1340 non-null   object 
 4   ห้อง                 125 non-null    object 
 5   ชั้น                 383 non-null    object 
 6   อาคาร                229 non-null    object 
 7   ซอย                  474 non-null    object 
 8   ถนน                  1115 non-null   object 
 9   หมู่                 162 non-null    object 
 10  ตำบล                 1343 non-null   object 
 11  อำเภอ                1343 non-null   object 
 12  จังหวัด              1343 non-null   object 
 13  รหสไปรษณีย์          1343 non-null   float64
 14  เบอร์โทรศัพท์        1314 non-null   object 
 15  เบอร์โทรสาร          232 non-null    object 

In [15]:
ct_license = ws_df['เลขใบอนุญาต'].value_counts().reset_index().sort_values(by='count',ascending=False)

In [16]:
ct_license.loc[ct_license['count']>1]

,เลขใบอนุญาต,count
0,1/2564,5
1,2/2564,5
2,6/2564,4
3,3/2564,4
4,4/2564,4
...,...,...
73,89/2563,2
74,154/2561,2
75,12/2561,2
76,12/2567,2


In [18]:
ct_license['distinct_type'] = np.nan

In [17]:
ws_df.loc[ws_df['เลขใบอนุญาต'] =='1/2564']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
1742,นางสาว พรพิศ นคเรศไอศูรย์,เมดพลัสคลินิกเวชกรรม,บ้านเลขที่900/31 ห้องเลขที่ี G5 ชั้น G อาคารเอ...,900/31,ห้องเลขที่ี G5,ชั้น G,อาคารเอสวีซิตี้ อาคาร 12,NaN,พระราม 3,NaN,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,10120.0,06 4556 5424,NaN,นางสาว พรพิศ นคเรศไอศูรย์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6400020.0,1/2564,อนุมัติ,NaN
1772,บริษัท โรงพยาบาล ซีจีเอช ลำลูกกา จำกัด,โรงพยาบาลซีจีเอช ลำลูกกา โรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่80/77-80 ถนน หมู่5 ตำบลลำลูกกา อ...,80/77-80,NaN,NaN,NaN,NaN,NaN,5,ลำลูกกา,ลำลูกกา,ปทุมธานี,12150.0,02 088 0888,02 088 0899,พลอากาศโท วันชัย ศิริเสรีวรรณ,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6400392.0,1/2564,อนุมัติ,NaN
1904,บริษัท พีลาทัส มารีน จำกัด,บริษัท พีลาทัส มารีน จำกัด,บ้านเลขที่84/1-4 ชั้น 1-4 อาคารยังเพลส แกรนด์...,84/1-4,NaN,ชั้น 1-4,อาคารยังเพลส แกรนด์ เลอ จาแด๊งน์,พหลโยธิน 37,NaN,NaN,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900.0,0 2930 1801-9,NaN,นาย วราวิช ฉิมตะวัน,จำหน่าย ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.),6400812.0,1/2564,อนุมัติ,NaN
1929,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บ้านเลขที่127 ห้อง 25.02 ชั้น 25 อาคาร เกษรทาว...,127,ห้อง 25.02,ชั้น 25,อาคาร เกษรทาวเวอร์,NaN,ราชดำริ,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2073 0224,NaN,นางสาว สุทัชชา อัจฉริยวงศ์กุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,6400866.0,1/2564,อนุมัติ,NaN
1935,องค์การเภสัชกรรม,องค์การเภสัชกรรม,บ้านเลขที่ 75/1 ซอย ถนนพระราม 6 หมู่ ตำบลทุ...,75/1,NaN,NaN,NaN,NaN,พระราม 6,NaN,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400.0,0 2203 8600-3,NaN,นาย วิฑูรย์ ด่านวิบูลย์,เพื่อผลิต ยส 2,6400868.0,1/2564,อนุมัติ,NaN


In [20]:
for i,v in ct_license.iterrows():
    license_id = v['เลขใบอนุญาต']
    ct_license.at[i, 'distinct_type'] = ws_df.loc[ws_df['เลขใบอนุญาต'] ==license_id]['ประเภทคำขอ'].nunique()

In [27]:
ct_license.loc[ct_license['count'].ne(ct_license['distinct_type'])]

,เลขใบอนุญาต,count,distinct_type


In [24]:
ct_license.loc[ct_license['count'].eq(ct_license['distinct_type'])]

,เลขใบอนุญาต,count,distinct_type
0,1/2564,5,5.0
1,2/2564,5,5.0
2,6/2564,4,4.0
3,3/2564,4,4.0
4,4/2564,4,4.0
...,...,...,...
1254,ควจ4-ร 47/2567,1,1.0
1255,จยส2-ร 40/2567,1,1.0
1256,นวจ2-บ 20/2567,1,1.0
1257,59/2567,1,1.0


In [13]:
ct_alltypes = wsf_df['ประเภทคำขอ'].value_counts().reset_index()

In [44]:
for i,v in ct_alltypes.iterrows():
    print(f"{i+1}, {v['ประเภทคำขอ']}")

1, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)
2, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)
3, ครอบครอง วจ2 สถานพยาบาล เอกชน
4, นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์
5, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)
6, ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
7, นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
8, ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือในกรณีเกิดเหตุฉุกเฉินในยานพาหนะที่ใช้ในการขนส่งสาธารณะระหว่างประเทศที่จดทะเบียนในราชอาณาจักร
9, ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์
10, ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
11, ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน
12, ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์
13, ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์
14, ครอ

In [43]:
wsf_df['ชื่อผู้รับอนุญาต'].nunique()

959

In [46]:
wsf_df['เลขใบอนุญาต'].nunique()

1272

In [48]:
wsf_df['ที่อยู่สถานที่'].nunique()

1103

In [47]:
wsf_df['ชื่อผู้ดำเนินกิจการ'].nunique()

1105

In [49]:
ct_alltypes

,ประเภทคำขอ,count
0,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,541
1,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,358
2,ครอบครอง วจ2 สถานพยาบาล เอกชน,147
3,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,93
4,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,62
5,ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรื...,41
6,นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเ...,33
7,ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือใ...,28
8,ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในป...,18
9,ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื...,12


In [50]:
ct_alltypes['ctd_mah_name'] = np.nan
ct_alltypes['ctd_เลขใบอนุญาต'] = np.nan
ct_alltypes['ที่อยู่สถานที่'] = np.nan
ct_alltypes['ชื่อผู้ดำเนินกิจการ'] = np.nan

In [51]:
for i,v in ct_alltypes.iterrows():
    name = v['ประเภทคำขอ']
    ct_alltypes.at[i, 'ctd_mah_name'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ชื่อผู้รับอนุญาต'].nunique()
    ct_alltypes.at[i, 'ctd_เลขใบอนุญาต'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['เลขใบอนุญาต'].nunique()
    ct_alltypes.at[i, 'ที่อยู่สถานที่'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ที่อยู่สถานที่'].nunique()
    ct_alltypes.at[i, 'ชื่อผู้ดำเนินกิจการ'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ชื่อผู้ดำเนินกิจการ'].nunique()

In [54]:
ct_alltypes.to_csv("C:/Users/user/Downloads/ws_pre_ctdalltypes.csv", index=False)

In [30]:
ct_alltypes['gr_types'] = np.nan

In [31]:
ct_alltypes

,ประเภทคำขอ,count,gr_types
0,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,541,NaN
1,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,358,NaN
2,ครอบครอง วจ2 สถานพยาบาล เอกชน,147,NaN
3,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,93,NaN
4,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,62,NaN
5,ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรื...,41,NaN
6,นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเ...,33,NaN
7,ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือใ...,28,NaN
8,ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในป...,18,NaN
9,ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื...,12,NaN


<h1>Cross-validate:</h1>

<h2>Distribution report</h2>

In [32]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
0,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205,2024-01-31,ศรีประจันต์,7205,0.25,125.00
1,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302,2024-01-31,กะทู้,8302,0.50,0.00
2,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005,2024-01-31,บ้านโป่ง,7005,0.50,306.00
3,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.25,116.25
4,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.50,478.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601,2024-12-31,เมืองนราธิวาส,9601,1.00,342.00
28175,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0

In [33]:
loc_ref_pvhos = dist_df.loc[dist_df['CustomerTypeName'] == 'คลินิก']['ReferCustomerName'].drop_duplicates()

In [48]:
loc_ref_pvhos_bkk = dist_df.loc[(dist_df['CustomerTypeName'] == 'คลินิก') & (dist_df['CustomerAddressProvinceName'] == 'กรุงเทพมหานคร')]['ReferCustomerName'].drop_duplicates()

In [49]:
loc_ref_pvhos_bkk = loc_ref_pvhos_bkk.reset_index()

In [37]:
isPresent = []
for name in loc_ref_pvhos_bkk:
    loc_sitename = ws_df.loc[ws_df['ชื่อสถานที่'].str.contains(name,regex=False)]
    if len(loc_sitename) >0:
        isPresent.append(True)
        continue
    loc_mahname = ws_df.loc[ws_df['ชื่อผู้รับอนุญาต'].str.contains(name,regex=False)]
    if len(loc_mahname) >0:
        isPresent.append(True)
        continue
    else:
        isPresent.append(False)

In [50]:
loc_ref_pvhos_bkk['isPresent'] = isPresent

In [64]:
loc_ref_pvhos_bkk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   index              226 non-null    int64   
 1   ReferCustomerName  226 non-null    object  
 2   isPresent          226 non-null    bool    
 3   CustomerTypeName   226 non-null    object  
 4   _merge             226 non-null    category
dtypes: bool(1), category(1), int64(1), object(2)
memory usage: 6.0+ KB


<h3>Validate join</h3>

In [53]:
dist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28179 entries, 0 to 28178
Data columns (total 32 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   ReportDisburseNarcoticBeYear              28179 non-null  float64
 1   ReportDisburseNarcoticMonth               28179 non-null  int64  
 2   ProductTypeName                           28179 non-null  object 
 3   MonthlyReportNarcoticName                 28179 non-null  object 
 4   MonthlyReportNarcoticBalanceForwardValue  28179 non-null  float64
 5   MonthlyReportNarcoticReceiveValue         28179 non-null  float64
 6   MonthlyReportNarcoticPaidValue            28179 non-null  float64
 7   MonthlyReportNarcoticRemainValue          28179 non-null  float64
 8   ReferCustomerName                         28179 non-null  object 
 9   CustomerAddressProvinceName               28179 non-null  object 
 10  CustomerTypeName                  

In [54]:
hos_types_df = dist_df[['ReferCustomerName','CustomerTypeName']].drop_duplicates().copy()

In [55]:
loc_ref_pvhos_bkk = loc_ref_pvhos_bkk.merge(
    hos_types_df,
    how = 'left',
    left_on = ['ReferCustomerName'],
    right_on = ['ReferCustomerName'],
    validate = '1:1',
    indicator = True
)

In [63]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent'] ==False].count()

index                28
ReferCustomerName    28
isPresent            28
CustomerTypeName     28
_merge               28
dtype: int64

In [65]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent'] ==True].count()

index                198
ReferCustomerName    198
isPresent            198
CustomerTypeName     198
_merge               198
dtype: int64

In [74]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent']==False]

,index,ReferCustomerName,isPresent,CustomerTypeName,_merge
13,234,ลาดกระบัง ซ.1 คลินิกเวชกรรม,False,คลินิก,both
18,285,คลินิกพัฒนา 25 การแพทย์เวชกรรม,False,คลินิก,both
26,405,คลินิกดุสิตาการแพทย์,False,คลินิก,both
35,494,คลินิกเวชกรรม บุญสิทธิการแพทย์,False,คลินิก,both
44,599,่คลินิกเวชกรรมแพทย์อิทธิพล,False,คลินิก,both
49,671,บลอสชั้นมคลินิกเวชกรรม,False,คลินิก,both
54,720,สหแพทย์หลังสวนสหคลินิก,False,คลินิก,both
60,798,คลินิก เวชกรรมพระราม 2,False,คลินิก,both
62,811,พาต้าโพลีคลินิกเวชกรรม,False,คลินิก,both
70,876,ทรานส์เซนต์คลินิกเวชกรรม,False,คลินิก,both


In [116]:
dist_df.loc[dist_df['ReferCustomerName'] == 'พาต้าโพลีคลินิกเวชกรรม']

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
811,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,bc4c15fe-b783-4e9e-a061-f83f84d41eee,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,0.25,0.0
812,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,30.0,8000.0,730.0,7300.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,969296c7-032f-4e53-b36f-0d10ed40efbc,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,1.00,730.0
813,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,075a18bc-a742-4460-a354-364d2f0b16e5,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,0.50,0.0
2712,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,5db6bcbd-76c5-477a-9f9f-89c3d59a1ac3,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,0.25,0.0
2713,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,7300.0,0.0,6317.0,983.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,836cfe4e-f08d-4518-b3f9-414c6d98b124,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,1.00,6317.0
2714,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,141dbf96-1c57-43b3-98d6-ac48792d3437,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,0.50,0.0
7075,2567.0,3,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,15/05/2024 13:17 น.,นาง จริยา บัวศรี,2024-05-15,2561137

In [107]:
loc_ref_pvhos_bkk.loc[216,'ReferCustomerName']

'คลินิกเวชกรรมเฉพาะทางศัลยกรรมตกแต่งและผิวหนัง เลเซอร์ พีเอไอ'

In [112]:
dist_df.loc[234]

ReportDisburseNarcoticBeYear                                                           2567.0
ReportDisburseNarcoticMonth                                                                 1
ProductTypeName                                               (วจ. 2) วัตถุออกฤทธิ์ในประเภท 2
MonthlyReportNarcoticName                   Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...
MonthlyReportNarcoticBalanceForwardValue                                                  0.0
MonthlyReportNarcoticReceiveValue                                                     10000.0
MonthlyReportNarcoticPaidValue                                                         1000.0
MonthlyReportNarcoticRemainValue                                                       9000.0
ReferCustomerName                                                 ลาดกระบัง ซ.1 คลินิกเวชกรรม
CustomerAddressProvinceName                                                     กรุงเทพมหานคร
CustomerTypeName                                            

In [117]:
ws_df[ws_df['อำเภอ'] =='บางพลัด']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
51,นาย เกรียงไกร อุรุโสภณ,คลินิกเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900630.0,14/2560 (วจ.2),อนุมัติ,NaN
52,นาย เกรียงไกร อุรุโสภณ,คลินิกเวชกรรมเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ3 สถานพยาบาล เอกชน,5900631.0,10/2560 (วจ.3),อนุมัติ,NaN
53,นาย เกรียงไกร อุรุโสภณ,คลินิกเวชกรรมเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ4 สถานพยาบาล เอกชน,5900632.0,11/2560 (วจ.4),อนุมัติ,NaN
150,นาย เกตุ สายเพ็ชร์,สายเพ็ชร์คลินิกเวชกรรม,บ้านเลขที่312 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,312,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางยี่ขัน,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 6626,NaN,นาย เกตุ สายเพ็ชร์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5901093.0,NaN,อนุมัติ,NaN
151,นาย เกตุ สายเพ็ชร์,สายเพ็ชร์คลินิกเวชกรรม,บ้านเลขที่312 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,312,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางยี่ขัน,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 6626,NaN,นาย เกตุ สายเพ็ชร์,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
220,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
221,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ4 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
222,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ3 สถานพยาบาล เอกชน,5901456.0,181/2560 (วจ.3),อนุมัติ,NaN
223,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ4 สถานพยาบาล เอกชน,5901457.0,286/2560 (วจ.4),อนุมัติ,NaN
349,บริษัท โรงพยาบาลสัตว์ทองหล่อ จำกัด,โรงพยาบาลสัตว์ทองหล่อ สาขาปิ่นเกล้า,บ้านเลขที่493 ซอย ถนนสิรินธร หมู่3 ตำบลบา...,493,NaN,NaN,NaN,NaN,สิรินธร,3,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2712 6301 ต่อ 11159,NaN,บริษัท โรงพยาบาลสัตว์ทองหล่อ จำกัด,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,0.0,NaN,อนุมัติ,NaN


In [109]:
ws_df.loc[586, 'ชื่อสถานที่']

'คลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่งและผิวหนัง เลเซอร์ พีเอไอ'

In [110]:
ws_df.loc[ws_df['ชื่อสถานที่'].str.contains('อรรถพร')]

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
678,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6100113.0,322/2561,อนุมัติ,NaN
1135,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6102637.0,ควจ4-ร 257/2562,อนุมัติ,NaN
2433,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6402731.0,จยส2-ร 212/2565,อนุมัติ,E
